# An advanced look at LLM Agents
## Notebook 3: Agents behind the hood
---

Previously, we relied on LangChain to to build the agent. However, this is not necessary, since an agent is nothing more than a fancy while loop. 

**The goal** 

With this notebook you will see what an agent is under the hood, and you can build it based on the LLM output

🌟 So ... let us begin!  

**Contents:**

1. [Exercise 1: New tool format](#1)
2. [Exercise 2: String vs function call response](#2)
3. [Exercise 3: Understanding the agent while-loop](#3)

In [ ]:
# Run this cell once, if you have not run the following command previously in the terminal. 
# Afterwards commenting it out  so it does not clutter your notebook.
!pip3 install -r requirements.txt

In [ ]:
from helper_functions.helper_functions import simple_description_formatter
from helper_functions.tools import my_own_wiki_tool, weather_tool
import pprint
from helper_functions.keys import client
import json


## The tools

When querying OpneAI's API directly tools are now called functions (see the API documentation on function calling [here](https://platform.openai.com/docs/assistants/tools/function-calling/quickstart)). Functions have to be passed in a specific JSON format, which we will explore below.


#### Exercise 1: New tool format <a id='1'></a>

Compile the cell below and check out the description of the first tool. How does it differ from the previous tool description?

In [ ]:
# load the tools and format. In plain OpenAI jargon they are called functions.
tools = [my_own_wiki_tool, weather_tool]
function_description = [simple_description_formatter(tool) for tool in tools]

# Store executable functions with their name in dictionary
available_functions = {tool.name: tool for tool in tools}


In [ ]:
pprint.pprint(function_description[0])

# compare this to the description directly obtained from the tool
print( #TODO: your code here 
    )

## The LLM response

The LLM can respond with two types of answers:
* a string that answers the question, 
* a function-call object, which contains information on which function to call with which arguments.

The second one can be used to execute function calls.

#### Exercise 2: String vs function call response <a id='2'></a>

Compile both questions, and compare the answers. Do you understand the difference?


In [ ]:
system_prompt = "You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner."

questions = ["what is the meaning of life?","How many people live in Paris?"]

for question in questions:
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

  
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
    )

  print(f"Question: {question}")
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")



## The Agent - a fancy while loop

While the LLM requests function calls we 
* **extract** the **name and arguments** to be called from the initial LLM response,
* **execute** the **function calls**,
* **store** the **output of the function** in the messages object,
* invoke the LLM again, until no function call are requested.

For more details, you can also check out this [OpenAI function calling guide](https://platform.openai.com/docs/guides/function-calling).

#### Exercise 3: Understanding the agent while-loop <a id='3'></a>

Change the question and investigate the output. Do you understand what you see?

In [11]:
question = "which city is bigger: Paris or Munich?"

messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  tools = function_description,
  messages=messages, 
  )

print('==== Initial LLM response ====')
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

# while the response requests function calls
while response.choices[0].message.tool_calls:
    
  # store response message with all function calls
  response_message = response.choices[0].message
  messages.append(response_message)

  # execute each tool individually
  for tool_call in response.choices[0].message.tool_calls:
    print('==== Function call ====')

    # function name and arguments
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(f'Calling function "{function_name}" with arguments {function_args}.')

    # execute function call 
    function_response = available_functions[function_name].invoke(function_args)
    print(f'Function call response:\n{function_response}\n')

    # append function response to messages
    messages.append({
        "tool_call_id":tool_call.id, 
        "role": "tool", 
        "name": function_name, 
        "content": function_response
    })
    
  # get a new response from LLM
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
  )

  print('== Intermediate LLM response ==')
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")

print('==== Final LLM response ====')
print("Question: ", question)
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

Function call response:
Page: Munich
Summary: Munich ( MEW-nik(h); German: München [ˈmʏnçn̩] ) is the capital and most populous city of the Free State of Bavaria, Germany. With a population of 1,589,706 inhabitants as of 29 February 2024, it is the third-largest city in Germany, after Berlin and Hamburg, and thus the largest which does not constitute its own state, as well as the 11th-largest city in the European Union. The city's metropolitan region is home to about 6.2 million people and the third largest metropolitan region by GDP in the European Union.
Straddling the banks of the river Isar north of the Alps, Munich is the seat of the Bavarian administrative region of Upper Bavaria, while being the most densely populated municipality in Germany with 4,500 people per km2. Munich is the second-largest city in the Bavarian dialect area, after the Austrian capital of Vienna.
The city was first mentioned in 1158. Catholic Munich strongly resisted the Reformation and was a political poin

🌟 Congratulations - you've finished the workshop